# Registered Companies in East Tamaki (New Zealand)


The main objective of this project is to scrape some specific information from the New Zealand Companies register website.

Our focus is majorly on the registered companies in East Tamaki.

The information that we are interested in are stated below:

        - The name of the company.
        - Registration number.
        - Shareholder names.
        - Shareholder address.
        - Percentage share of each shareholder.
        - Role of shareholder.
        
The major packages used to achieve this task is Selenium and BeautifulSoup.









In [3]:

#import all necessary packages for this process

import selenium

import requests

import time

from selenium import webdriver

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support.select import Select

from bs4 import BeautifulSoup as soup

import pandas as pd

import numpy as np

from time import time

from time import sleep

In [4]:
#load the webdriver to drive the page

driver = webdriver.Chrome("C:/Users/PELUMI/Downloads/chromedriver_win32/chromedriver.exe")

In [5]:
#Using the get function, we access the landing page we want to work from

driver.get("https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search?mode=standard&type=entities&advancedPanel=true&entityTypes=ALL&entityStatusGroups=ALL")

In [6]:



#input the keyword into the search box

searchbox =driver.find_elements_by_xpath("//input[@name='addressKeyword' and @class ='text']")[0]
searchbox.click()
keyword = "East Tamaki"
searchbox.send_keys(keyword)


##clicking the dropdown button
dropdown_button = driver.find_elements_by_xpath('//*[@id="entityStatusGroups_1"]')[0]
dropdown_button.click()


###choosing the registered option
reg_option = driver.find_elements_by_xpath('//*[@id="aEntityStatusGroups"]/div[2]/div[2]/label')[0]
reg_option.click()


###Doing the search
search = driver.find_element_by_xpath('//*[@id="advancedSearchButton"]')
search.click()

sleep(5)


In [ ]:

#lists
number_of_shares = []
percentage_share = []
shareholders = []
shareholders_address = []
shareholders_roles = []
company = []

#iterating through the first 5 pages that we want to scrape, each page contains about 15 companies listed

start_time = time()

request = 0

for i in np.arange(0,61,15):
    
    link = 'https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search?q=&start={}&limit=15&entitySearch=&addressKeyword=East%20Tamaki&postalCode=&incorpFrom=&incorpTo=&country=&addressType=&advancedPanel=true&mode=advanced&sf=&sd=&entityTypes=ALL&entityStatusGroups=REGISTERED&addressTypes=ALL'.format(i)
    
    page = driver.get(link)
    
    sleep(5)
    
    elasped_time = time() - start_time
    
    request += 1
    
    print('request: {}; frequency: {} request/s'.format(request, request/elasped_time))
    
    if request > 5:
        
        print('Number of request greater than expected')
        
        break 
    count = 0

    start_time = time()


    links = len(driver.find_elements_by_class_name("entityName"))
    
#iterating through each companies listed on each page

    for link in range (links):

        companies = driver.find_elements_by_class_name("entityName")[link]

        companies.click()

        sleep(3)
        
#clicking the view as single page option

        single_page_view = driver.find_element_by_xpath('//*[@id="showAllDetails"]').click()


#the scraping process begins here as we parse each of the companies page through beautifulSoup to get the information we want


        current_page = driver.page_source

        page = soup(current_page, 'html.parser')

        elements = page.find_all('div', class_='allocationDetail')
        
#we loop through all the informations we want and we store them in a list

        for element in elements:

            company_name = page.find('div', class_='entity').h1.text.strip()
            share_number = element.find('span', class_='allocationNumber').text
            number_of_shares.append(share_number)
            share_percent = element.find('span', class_='shareLabel').text.replace('\xa0','').replace('(','').replace(')','').replace('shares','')
            percentage_share.append(share_percent)
            company_label = element.find_all('div',class_='labelValue')[0].text.strip()
            shareholders.append(company)
            address = element.find_all('div',class_='labelValue')[1].text.strip()
            shareholders_address.append(address)
            roles = element.find('span', class_= 'label directorLabel')

            
#Since some of the shareholders have no roles, we create a conditional statement to handle that exception

            if roles == None:
                shareholders_roles.append(roles)

            else:
                roles = element.find('span', class_= 'label directorLabel').text
                shareholders_roles.append(roles)


            company_name = [company_name]*len(number_of_shares)
            company.append(company_name)

    
        elasped_time = time() - start_time

        count += 1
        
#this print statement is just to monitor the page scraping process and it could be removed

        #print('link: {}; frequency: {} link/s'.format(count,elasped_time))


        driver.execute_script("window.history.go(-2)")

        sleep(3)


In [21]:
company_data = pd.DataFrame(list(zip(company,shareholders, percentage_share,number_of_shares,shareholders_address,shareholders_roles)), columns= 
             ["company", "shareholder_name", "percentage_share","number_of_shares", "address", "roles"])

In [40]:
#Removing unwanted characters from the data

company_data['company'] = company_data['company'].astype(str).str.replace('[','').str.replace(']','')

company_data['shareholder_name'] = company_data['shareholder_name'].astype(str).str.replace('[','').str.replace(']','')
company_data['address'] = company_data['address'].astype(str).str.replace('\n','')


In [41]:
company_data

,company,shareholder_name,percentage_share,number_of_shares,address,roles
0,'NAND CORPORATION LIMITED (2466968) Registered','NAND CORPORATION LIMITED (2466968) Registere...,100.00%,1,"40 Redoubt Road, Manukau City, New Zealand",None
1,'NEW ZEALAND ORGANIC MILK LIMITED (3275068) R...,'NAND CORPORATION LIMITED (2466968) Registere...,100.00%,1,"25 Hayf, East Tamaki, Manukau, 2013, New Zealand",Director:
2,'A 2 Z NZ POWERCOM LIMITED (836853) Registere...,'NAND CORPORATION LIMITED (2466968) Registere...,50.00%,1,"443 East Tamaki Road, East Tamaki, Auckland, 2...",None
3,'A 2 Z NZ POWERCOM LIMITED (836853) Registere...,'NAND CORPORATION LIMITED (2466968) Registere...,50.00%,2,"443 East Tamaki Road, East Tamaki, Auckland, 2...",Director:
4,"""ABDUL'S TRUCK AND TRAILER LIMITED (5365241) ...",'NAND CORPORATION LIMITED (2466968) Registere...,100.00%,1,"712a Massey Road, Mangere, Auckland, 2022, New...",Director:
...,...,...,...,...,...,...
91,'FIBRE OPTIONS NZ LIMITED (2298947) Registere...,'NAND CORPORATION LIMITED (2466968) Registere...,24.14%,3,"2/43 Robertson Road, Mangere, Auckland, New Ze...",None
92,'FIBRE OPTIONS NZ LIMITED (2298947) Registere...,'NAND CORPORATION LIMITED (2466968) Registere...,6.90%,4,"12 Lucia Glade, Meadowbank, Auckland, 1072, Ne...",None
93,'RAJ MACHINE RENTAL LIMITED (5871597) Registe...,'NAND CORPORATION LIMITED (2466968) Registere...,50.00%,1,"371 East Tamaki Road, East Tamaki, Auckland, 2...",None
94,'RAJ MACHINE RENTAL LIMITED (5871597) Registe...,'NAND CORPORATION LIMITED (2466968) Registere...,50.00%,2,"371 East Tamaki Road, East Tamaki, Auckland, 2...",Director:


In [42]:
#Converting the Dataframe to a csv file

company_data.to_csv("Company data_EastTamaki.csv")
